In [1]:
import numpy as np
from tensorflow import keras
from keras.utils import to_categorical
from keras.datasets import cifar10
from keras.applications.nasnet import NASNetLarge
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, Input, Flatten, Dropout, BatchNormalization, Conv2D, MaxPooling2D, concatenate, GlobalAveragePooling2D
from keras.models import Sequential, Model

2023-07-30 08:36:42.778017: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-30 08:36:42.971264: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [12]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)
x_train, x_test = x_train / 255., x_test / 255.
y_train, y_test = to_categorical(y_train, 10), to_categorical(y_test, 10)

In [13]:
model_nas = NASNetLarge(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
model_nas.trainable = False

In [14]:
model_vgg = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
model_vgg.trainable = True

In [15]:
model_nas_out = model_nas.output
model_nas_out = GlobalAveragePooling2D()(model_nas_out)
model_vgg_out = model_vgg.output
model_vgg_out = GlobalAveragePooling2D()(model_vgg_out)
x = concatenate([model_nas_out, model_vgg_out])
x = Dense(10, activation='softmax', kernel_regularizer='l1')(x)

In [16]:
model = Model(inputs=[model_nas.input, model_vgg.input], outputs=x)
model.compile(optimizer='adamax', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
res = model.fit([x_train, x_train], y_train, batch_size=32, epochs=10, validation_data=([x_test, x_test], y_test))

Epoch 1/10
1563/1563 [==============================] - 416s 260ms/step - loss: 0.4429 - accuracy: 0.2440 - val_loss: 0.2850 - val_accuracy: 0.4806
Epoch 2/10
1563/1563 [==============================] - 408s 261ms/step - loss: 0.2119 - accuracy: 0.6638 - val_loss: 0.1903 - val_accuracy: 0.7350
Epoch 3/10
1563/1563 [==============================] - 410s 262ms/step - loss: 0.1584 - accuracy: 0.7982 - val_loss: 0.1569 - val_accuracy: 0.8109
Epoch 4/10
1563/1563 [==============================] - 410s 262ms/step - loss: 0.1334 - accuracy: 0.8498 - val_loss: 0.1454 - val_accuracy: 0.8283
Epoch 5/10
1563/1563 [==============================] - 408s 261ms/step - loss: 0.1149 - accuracy: 0.8876 - val_loss: 0.1441 - val_accuracy: 0.8391
Epoch 6/10
1563/1563 [==============================] - 410s 262ms/step - loss: 0.0996 - accuracy: 0.9160 - val_loss: 0.1521 - val_accuracy: 0.8362
Epoch 7/10
  95/1563 [>.............................] - ETA: 5:58 - loss: 0.0895 - accuracy: 0.9365